In [1]:
import SimpleITK as sitk
import numpy as np
import nibabel as nib
from scipy.ndimage import affine_transform
from scipy import ndimage
from pathlib import Path

def registration():
    fixed_img_file = 'data/train/Patient_27/GT2.nii.gz' 
    moving_img_file = 'data/train/Patient_27/GT.nii.gz'
    transform_file = 'registration.tfm'

    fixed_img = sitk.ReadImage(fixed_img_file, sitk.sitkFloat32)
    fixed_img = sitk.GetArrayFromImage(fixed_img)
    fixed_img = (fixed_img == 2).astype(np.float32)
    fixed_img = sitk.GetImageFromArray(fixed_img)

    moving_img = sitk.ReadImage(moving_img_file, sitk.sitkFloat32)
    moving_img = sitk.GetArrayFromImage(moving_img)
    moving_img = (moving_img == 2).astype(np.float32)
    moving_img = sitk.GetImageFromArray(moving_img)

    registration = sitk.ImageRegistrationMethod()
    registration.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration.SetMetricSamplingStrategy(registration.RANDOM)
    registration.SetMetricSamplingPercentage(0.02)
    registration.SetInterpolator(sitk.sitkNearestNeighbor)
    registration.SetOptimizerAsGradientDescent(learningRate=0.8, numberOfIterations=100, convergenceMinimumValue=1e-15, convergenceWindowSize=1)
    registration.SetOptimizerScalesFromPhysicalShift()
    registration.SetInitialTransform(sitk.CenteredTransformInitializer(fixed_img, moving_img, sitk.AffineTransform(3), True))
    registration.SetShrinkFactorsPerLevel(shrinkFactors=[4,2,1])
    registration.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
    registration.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
    
    transform = registration.Execute(fixed_img, moving_img)
    sitk.WriteTransform(transform, transform_file)

In [2]:
# Step 2: Apply affine transformation using SciPy
def transformation(moving_image):
   
    # these come from the registration.ftm file
    parameters = [1, 0, 0, 0, 1, 0, 0, 0, 1, 0.4104296426513123, 0.1911134048201859 ,0.00178584400418913]
    #Insight Transform File V1.0
    #Transform 0
    #Transform: AffineTransform_double_3_3
    #Parameters: 1 0 0 0 1 0 0 0 1 0.4104296426513123 0.1911134048201859 0.00178584400418913
    #FixedParameters: 24.630227810960996 -205.9763013594433 -371.5123181176971

    affine_matrix = np.array([
        [parameters[0], parameters[1], parameters[2], parameters[9]],
        [parameters[3], parameters[4], parameters[5], parameters[10]],
        [parameters[6], parameters[7], parameters[8], parameters[11]],
        [0, 0, 0, 1]
    ])
    print(affine_matrix)

    rotation_scale = affine_matrix[:3, :3]
    
    moving_image_data = np.array(moving_image)
    print(moving_image_data)
    transformed_image_data = affine_transform(moving_image_data, rotation_scale, offset=affine_matrix[:3, 3], order=0)

    object_center = ndimage.center_of_mass(transformed_image_data)
    volume_center = np.array(transformed_image_data.shape) // 2

    shift = volume_center - np.array(object_center)
    print('shift!', shift)
    shifted_volume = ndimage.shift(transformed_image_data, shift, order=0)

    volume = ndimage.rotate(shifted_volume,-shift[1], axes=(0,1), reshape=False, order=0)

    return ndimage.shift(volume, -shift, order=0)


def save_image(transformed_image_data, reference_image_path, output_image_path):
    reference_image = nib.load(reference_image_path)
    reference_affine = reference_image.affine
    reference_header = reference_image.header
    
    reference_image_data = reference_image.get_fdata()
    reference_image_data[reference_image_data == 2] = 0
    reference_image_data[transformed_image_data == 1] = 2
    transformed_image_data = reference_image_data

    transformed_image_nifti = nib.Nifti1Image(transformed_image_data.astype(np.uint8), reference_affine, reference_header)
    nib.save(transformed_image_nifti, output_image_path)


fixed_img_file = 'data/train/Patient_27/GT2.nii.gz' 
moving_img_file = 'data/train/Patient_27/GT.nii.gz'
# Load the fixed and moving images
fixed_image = nib.load(fixed_img_file).get_fdata()
fixed_image = (fixed_image == 2).astype(np.float32)

moving_image = nib.load(moving_img_file).get_fdata()
moving_image = (moving_image == 2).astype(np.float32)

output_image_path = 'transformed_image.nii.gz'

registration()

transformed_image_data = transformation(moving_image)
summing = (transformed_image_data == fixed_image).sum() / fixed_image.size

print(f"Overlap: {round(summing*100, 2)}%")


[[1.         0.         0.         0.41042964]
 [0.         1.         0.         0.1911134 ]
 [0.         0.         1.         0.00178584]
 [0.         0.         0.         1.        ]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. .

In [4]:
import os
base_folder = 'data/segthor_train/train'
base_folder = 'data/train/'

for patient_id in range(2, 41):  
    patient_folder = f'Patient_{patient_id:02d}'
    input_seg_file = os.path.join(base_folder, patient_folder, 'GT.nii.gz')
    # print(input_seg_file)#data/train/Patient_01/GT.nii.gz

    moving_image = nib.load(input_seg_file).get_fdata()
    moving_image = (moving_image == 2).astype(np.float32)

    transformed_image = transformation(moving_image)
    
    transformed_image_filename = 'transformed_GT_manual.nii.gz'
    transformed_image_path = os.path.join(base_folder, patient_folder, transformed_image_filename)

    save_image(transformed_image, input_seg_file, transformed_image_path)
    
    # Check if the original GT file exists
    if os.path.exists(input_seg_file):
        # Rename the original GT file to GT_original.nii.gz
        original_gt_file = os.path.join(base_folder, patient_folder, 'GT_original.nii.gz')
        os.rename(input_seg_file, original_gt_file)
        print(f"Renamed {input_seg_file} to {original_gt_file}")

        new_gt_file = os.path.join(base_folder, patient_folder, 'GT.nii.gz')
        os.rename(transformed_image_path, new_gt_file)

[[1.         0.         0.         0.41042964]
 [0.         1.         0.         0.1911134 ]
 [0.         0.         1.         0.00178584]
 [0.         0.         0.         1.        ]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. .

KeyboardInterrupt: 